# Amazon EC2 (Elastic Compute Cloud)

<img src="../_assets/aws_service_icons/ec2.svg" width="80" alt="Amazon EC2">

## Goals
- Understand what **Amazon EC2** is (and what it is not).
- Know what EC2 is **practically used for**.
- See a minimal **AWS SDK** pseudo-code workflow (no execution).


## Prerequisites
- Basic AWS concepts: **region**, **IAM**, **VPC**.
- Basic networking: IPs, ports, security groups, SSH.
- Familiarity with the idea of a "virtual machine".


## What EC2 is
**Amazon EC2** is AWS's on-demand **compute** service for running virtual servers called **instances**.

With EC2 you choose:
- An **AMI** (Amazon Machine Image): the OS + preinstalled software baseline.
- An **instance type**: the CPU/RAM/network/GPU capacity (for example `t3.micro`, `m7i.large`, `g5.xlarge`).
- **Networking**: which VPC/subnet the instance lives in and which **security groups** (firewall rules) apply.
- **Storage**: typically **EBS volumes** (virtual disks) attached to the instance.

EC2 is "infrastructure" level compute. You are responsible for the OS and most lifecycle tasks (patching, hardening, scaling patterns), unless you layer additional managed services on top.


## What it is practically used for
Common real-world uses:
- **Hosting applications**: web servers, APIs, background workers.
- **Batch/ETL jobs**: scheduled processing, one-off data backfills.
- **Machine learning workloads**: model training on CPU/GPU instances, custom inference servers.
- **Running custom software stacks**: anything that needs full OS control (custom drivers, specialized binaries).
- **Self-managed services**: when you intentionally operate your own database/cache/queue (often for legacy or special requirements).

Typical supporting building blocks:
- **Auto Scaling Groups** for capacity management.
- **Load Balancers** (ALB/NLB) for traffic distribution.
- **CloudWatch** for metrics/logs/alerts.


## Core concepts (minimum you should know)
- **AMI**: image used to boot the instance.
- **Instance type**: sizing (CPU/RAM/GPU).
- **Subnet + Security Group**: where it runs and what inbound/outbound traffic is allowed.
- **Key pair / SSM**: how you access the instance (SSH keys or AWS Systems Manager).
- **IAM role (instance profile)**: how code on the instance accesses AWS APIs without hardcoding credentials.
- **Tags**: metadata for cost allocation and automation.


## Using EC2 with the AWS SDK (pseudo-code)
Below is a **conceptual** example using the AWS SDK for Python (**boto3**). Values like AMI IDs, subnets, and security groups are placeholders.

```python
# PSEUDO-CODE (do not execute)
import boto3

session = boto3.Session(profile_name="my-profile", region_name="us-east-1")
ec2 = session.client("ec2")

# 1) Launch an instance
resp = ec2.run_instances(
    ImageId="ami-0abc1234def567890",  # pick an AMI for your region
    InstanceType="t3.micro",
    MinCount=1,
    MaxCount=1,
    SubnetId="subnet-0123456789abcdef0",
    SecurityGroupIds=["sg-0123456789abcdef0"],
    KeyName="my-ssh-keypair",  # or use SSM instead of SSH
    TagSpecifications=[
        {
            "ResourceType": "instance",
            "Tags": [{"Key": "Name", "Value": "ckc-ec2-example"}],
        }
    ],
)

instance_id = resp["Instances"][0]["InstanceId"]

# 2) Wait until it's running
ec2.get_waiter("instance_running").wait(InstanceIds=[instance_id])

# 3) Fetch details (public IP/DNS depends on subnet + settings)
desc = ec2.describe_instances(InstanceIds=[instance_id])

# 4) Clean up when done (or stop if you plan to reuse)
ec2.terminate_instances(InstanceIds=[instance_id])
```


## Pitfalls & quick tips
- Prefer **IAM roles** on instances; avoid long-lived access keys on disk.
- Lock down **security groups** (least privilege). Avoid `0.0.0.0/0` SSH unless you truly need it.
- Remember to **stop/terminate** instances and delete unused EBS volumes to avoid surprise costs.
- Consider higher-level services when you don't need OS control (ECS/EKS/Lambda, managed databases).


## References
- AWS EC2 documentation
- boto3 EC2 client reference
